# Projeto K-mis
## Estudante: Gabriel Dias

## Descrição do problema

Para modelar o problema da máxima intersecção dos k-subconjuntos foram feitos os modelos _MEB - V_  e _MEB - L_. Para solucionar o problema, o primeiro faz mão de variáveis $y_{u}$, $z_{u}$, $z_{v}$, de tal forma que $y_{u}$ representa a quantidade de vértices vizinhos de um vértice $u \in L$ que pertencem a solução (ou seja, a quantidade de elementos de um subconjunto $u$ que pertencem a máxima intersecção dos k-subconjuntos), enquanto as variáveis $z_{u}$ e $z_{v}$ são de cunho _booleano_ e indicam se um vértice está contido ou não na solução, de tal forma que $u \in L$ e $v \in R$.

O modelo _MEB-L_ faz mão das mesmas variáveis, mas agora elas estão definidas para partições diferentes, a variável $y_{v}$ agora denota a quantidade de vértices vizinhos de um vértice $v \in R$ que pertencem a solução do k-mis (ou seja, a quantidade de subconjuntos da partição L que fazem parte dos $k$ subconjuntos do $k-mis$ que incluem um vértice $v \in R$). Enquanto isto, a variável $z_{u}$ mantém a mesma lógica, mas não faz uso de uma variável $z_{v}$

> Observação: Teoricamente, não existe destinção entre variável $z_{u}$ e variável $z_{v}$, pois ela é apenas uma única variável $z_{w}$ que podemos definir a qual partição o vértice $w$ pertence. No entanto, ao explicar este problema, é mais didático fazer distinção entre $z_{u}$ e $z_{v}$, por isso que durante a descrição do problema, esta distinção foi feita. 

## Descrição da implementação do modelo MEB-V

Para modelar as variáveis $z_{u}$ e $z_{v}$, ao  invés de criar duas variáveis diferentes para exprimir do modelo $MEB-V$, foi notado que a modelagem do k-mis ficaria mais fácil se fosse utilizado apenas uma variável $z$. Esta variável $z$ é representada através de um dicionário do tipo `vértice : valor de pertinência`, de tal forma que as chaves do primeira metade do dicionário dizem respeito aos vértices pertencentes a L (os subconjuntos) e a segunda metade das chaves do dicionário dizem respeito aos vértices pertencentes a R (os elementos). Para modelar a variável $y_{u}$ também foi feito um dicionário. Em relação a implementação da função objetiva e das restrições, elas são iguais ao do artigo apresentado.

## Descrição da implementação do modelo MEB-L

A lógica de modelagem do modelo MEB-L é praticamente igual ao do modelo anterior, mas é claro, com restrições diferentes e com a função objetiva agora calculando o produto entre $\frac{1}{k}$ e o somatório das variáveis $y_{v}, u \in R$, ao invés de ser o produto entre $\frac{1}{k}$ e o somatório das variáveis $y_{u}, u \in L$, além de variáveis $z$ e $y$ definidas sobre partições diferentes, é claro.

## Comparação de desempenho dos modelos

Os modelos apresentados foram comparados tendo em mente o tamanho das partições L e R e a desensidade do grafo. De tal forma que $|L| = |R|$ de tal forma que o tamanho das partições testadas estão definidas no conjunto ${10, 20, 30, 40, 50}$ e o conjunto de probabilidade de criação de arestas testadas corresponde a {0.5, 0.8} - na tabela este campo está representado com densidade. $K$ está fixado para ser igual 3. Todos os testes foram feitos em um notebook _HP 250 G8_
com processador _Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz 1.50 GHz_ com _16 GB de RAM_ rodando o sistema operacional _Windows 11_.

Uma tabela com os resultados obtidos encontrasse abaixo logo abaixo do output do programa. Em relação a ela, considere sempre que os resultados apresentados na já consideram k=3 e que o _tempo está medido em segundos_

In [1]:
import networkx as nx
from networkx.algorithms import bipartite
from ortools.linear_solver import pywraplp
import time

def MEB_V(G,L,R,k):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    z = {u: solver.BoolVar(f'z_{u}') for u in L+R}
    bigNumber = 1e9
    y = {u: solver.IntVar(0,bigNumber, f'y_{u}') for u in L}
    
    #função objetiva
    solver.Maximize(1/k * solver.Sum(y[u] for u in L))
    
    #restrição I
    for u in L:
        naoVizinhos = set(R) - set(G.neighbors(u))
        for v in naoVizinhos:
            solver.Add(z[u] + z[v] <= 1, f'R1_{u}_{v}')
    
    #restrição II
    for u in L:
        solver.Add(y[u] <= G.degree[u]*z[u], f'R2_{u}')
    
    #restrição III
    for u in L:
        solver.Add(y[u] <= solver.Sum(z[v] for v in G.neighbors(u)), f'R3_{u}')
    
    #restrição IV
    solver.Add(solver.Sum(z[u] for u in L) == k)
    
    #solver.parameters.max_time_in_seconds = 1800.0 - não tava dando certo
    solver.SetTimeLimit(1800 * 1000) #1800 milisegundos

    startTime = time.time()
    status = solver.Solve()
    endTime = time.time()

    timeTaken = endTime - startTime
        
    return (solver.Objective().Value(), [f"y[{u}] = {y[u].solution_value()}" for u in L], [f'z[{u}] =  {z[u].solution_value()}' for u in L+R],timeTaken,status)

def MEB_L(G,L,R,k):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    y = {v: solver.NumVar(0, solver.infinity(), f'y_{v}') for v in R}
    z = {u: solver.BoolVar(f'z_{u}')  for u in L}
    
    #função objetiva
    solver.Maximize(1/k * solver.Sum(y[v] for v in R))
    
    #restrição 7
    for v in R:
        solver.Add(y[v] <= solver.Sum(z[u] for u in G.neighbors(v)), f'R7_{v}')
    
    #restrição 8:
    for u in L:
        nao_vizinhos = set(R) - set(G.neighbors(u))
        for v in nao_vizinhos:
            solver.Add(y[v] <= (1-z[u])*G.degree(v), f'R8_{u}_{v}')
    
    #restriction 9
    solver.Add(solver.Sum(z[u] for u in L) == k, 'R9')

    #solver.parameters.max_time_in_seconds = 1800.0 - não tava dando certo
    solver.SetTimeLimit(1800 * 1000) #1800 milisegundos
    
    startTime = time.time()
    status = solver.Solve()
    endTime = time.time()

    timeTaken = endTime - startTime

    #if status == pywraplp.Solver.OPTIMAL:
    return (solver.Objective().Value(),[f'y[{v}] = {y[v].solution_value()}' for v in R], [f'z[{u}] = {z[u].solution_value()}' for u in L], timeTaken, status)

def showStatistics(meb_timesSparse,meb_timesDense,mebName):
    ###################################################
    mediaSparseMEB=0
    mediaDenseMEB=0
    mediaMEB = 0

    for i in meb_timesSparse:
        mediaSparseMEB += i
    mediaSparseMEB = mediaSparseMEB/len(meb_timesSparse)
    
    for i in meb_timesDense:
        mediaDenseMEB += i
    mediaDenseMEB = mediaDenseMEB/len(meb_timesDense)

    mediaMEB = (mediaSparseMEB + mediaDenseMEB)/2

    print(f"-=-=-=Estatísticas do {mebName}-=-=-=")

    print(f"Média de tempo em grafos esparsos: {mediaSparseMEB/60} minutos -> {mediaSparseMEB} segundos")
    print(f"Média de tempo em grafos densos: {mediaDenseMEB/60} minutos -> {mediaDenseMEB} segundos")
    print(f"Média de tempo geral: {mediaMEB/60} minutos -> {mediaMEB} segundos")

def wrapper(tamanhoParticoes, probabilidades, kmis):
    mebV_timesSparse = []
    mebV_timesDense = []
    mebV_allTimes = []

    mebL_timesSparse = []
    mebL_timesDense = []

    for tamanho in tamanhoParticoes:
        mebV_timesSparse2 = []
        mebV_timesDense2 = []
        mebV_allTimes2 = []

        mebL_timesSparse2 = []
        mebL_timesDense2 = []
        for probabilidade in probabilidades :
            
            G = bipartite.random_graph(tamanho,tamanho,probabilidade)
            #esta condição foi posta para evitar o erro
            #"AmbiguousSolution: Disconnected graph: Ambiguous solution for bipartite sets."
            #que acontecia aleatoriamente
            while not nx.is_connected(G):
                G = None
                G = bipartite.random_graph(tamanho,tamanho,probabilidade)
            L, R = bipartite.sets(G)
            L = list(L)
            R = list(R)

            print("-=-=-=-PARTIÇÃO L-=-=-=-=")
            neighborhood_dict = {node: list(G.neighbors(node)) for node in G.nodes()}
            counter = 0
            for node in neighborhood_dict:
                if counter == len(neighborhood_dict)//2:
                    print("-=-=-=-=PARTIÇÃO R-=-=-=-=")
                print(node,":", neighborhood_dict[node])
                counter += 1
            print("densidade do grafo gerado:",nx.density(G))
            
            for k in kmis :
                resultsMEB_V = MEB_V(G,L,R,k)
                resultsMEB_L = MEB_L(G,L,R,k)

                if (probabilidade == 0.5):
                    mebV_timesSparse.append(resultsMEB_V[3])
                    mebL_timesSparse.append(resultsMEB_L[3])
                    mebV_timesSparse2.append(resultsMEB_V[3])
                    mebL_timesSparse2.append(resultsMEB_L[3])
                else:
                    mebV_timesDense.append(resultsMEB_V[3])
                    mebL_timesDense.append(resultsMEB_L[3])
                    mebV_timesDense2.append(resultsMEB_V[3])
                    mebL_timesDense2.append(resultsMEB_L[3])
                    
                
                print("-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=")
                print(f"Valor de k= {k}")
                print(f"Valor da probabilidade: {probabilidade}")
                print(f"Tamanho das partições L e R: {tamanho}")
                print("-=-=-=-=-=-=MEB-V-=-=-=-=-=")
                if ((resultsMEB_V)[-1] == pywraplp.Solver.OPTIMAL):
                    print("Solução ótima encontrada:")
                else:
                    print("Solução viável encontrada")
                print(f"Resultado do K-mis: {resultsMEB_V[0]}")
                print(f"Tempo levado: {resultsMEB_V[3]/60} minutos -> {resultsMEB_V[3]} segundos")
                print("Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução")
                for i in range(len(resultsMEB_V[1])):
                    if (resultsMEB_V[1][i] != '0.0'):
                        print(resultsMEB_V[1][i])
                print("Vértices de L que pertecem a solução do k-mis")
                for i in range(len(resultsMEB_V[2])//2):
                    if (resultsMEB_V[2][i][-3:] == '1.0'):
                        print(resultsMEB_V[2][i])
                print("Vértices de R que pertencem a solução do kmis")
                for i in range(len(resultsMEB_V[2])//2, len(resultsMEB_V[2])):
                    if (resultsMEB_V[2][i][-3:] == '1.0'):
                        print(resultsMEB_V[2][i])
                        
                print("-=-=-=-=-=-=MEB-L-=-=-=-=-=")
                if (resultsMEB_L[-1] == pywraplp.Solver.OPTIMAL):
                    print("Solução ótima encontrada:")
                else :
                    print("Solução viável encontrada:")
                    
                print(f"quantidade de elementos no {tamanho}-mis {resultsMEB_L[0]}")
                print(f"Tempo levado: {resultsMEB_L[3]/60} minutos -> {resultsMEB_L[3]} segundos")
                print("Quantidade de vizinhos de um vértice 'v' em R que pertencem a solução")
                for i in range(len(resultsMEB_L[1])):
                    if (resultsMEB_L[1][i][-3:] != '0.0'):
                        print(resultsMEB_L[1][i])
                print("Vertices de L que pertencem a solução do kmis")
                for i in range(len(resultsMEB_L[2])):
                    if (resultsMEB_L[2][i][-3:] == '1.0'):
                        print(resultsMEB_L[2][i])

                del G
        print(f"-=-=-=-=-=Estatísticas para tamanho '{tamanho}'-=-=-=-=-=")
        showStatistics(mebV_timesSparse2,mebV_timesDense2,"MEB-V")
        showStatistics(mebL_timesSparse2,mebL_timesDense2,"MEB-L")
                
    print("-=-=-=-=-=Estatísticas gerais-=-=-=-=-=")

    showStatistics(mebV_timesSparse,mebV_timesDense,"MEB-V")
    showStatistics(mebL_timesSparse,mebL_timesSparse, "MEB-L")


def main():
    tamanhoParticoes = [10,20,30,40,50]
    probabilidades = [0.5,0.8]
    kmis = [3]
    wrapper(tamanhoParticoes, probabilidades, kmis)

if __name__ == "__main__":
    main()

-=-=-=-PARTIÇÃO L-=-=-=-=
0 : [10, 12, 14, 17]
1 : [10, 12, 16, 19]
2 : [14, 15, 16, 17, 18, 19]
3 : [12, 15, 18]
4 : [10, 11, 14, 16, 17]
5 : [10, 12, 13, 15, 17, 18, 19]
6 : [10, 11, 12, 13, 14, 15, 17, 19]
7 : [10, 14, 16, 19]
8 : [10, 17, 19]
9 : [12, 13, 15, 16, 17]
-=-=-=-=PARTIÇÃO R-=-=-=-=
10 : [0, 1, 4, 5, 6, 7, 8]
11 : [4, 6]
12 : [0, 1, 3, 5, 6, 9]
13 : [5, 6, 9]
14 : [0, 2, 4, 6, 7]
15 : [2, 3, 5, 6, 9]
16 : [1, 2, 4, 7, 9]
17 : [0, 2, 4, 5, 6, 8, 9]
18 : [2, 3, 5]
19 : [1, 2, 5, 6, 7, 8]
densidade do grafo gerado: 0.2578947368421053
-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=
Valor de k= 3
Valor da probabilidade: 0.5
Tamanho das partições L e R: 10
-=-=-=-=-=-=MEB-V-=-=-=-=-=
Solução ótima encontrada:
Resultado do K-mis: 4.0
Tempo levado: 0.004006528854370117 minutos -> 0.24039173126220703 segundos
Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução
y[0] = 0.0
y[1] = 0.0
y[2] = 0.0
y[3] = 0.0
y[4] = 0.0
y[5] = 4.0
y[6] = 4.0
y[7] = 0.0
y[8] = 0.0
y[9] = 4.

-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=
Valor de k= 3
Valor da probabilidade: 0.8
Tamanho das partições L e R: 30
-=-=-=-=-=-=MEB-V-=-=-=-=-=
Solução ótima encontrada:
Resultado do K-mis: 22.999999999999993
Tempo levado: 0.008276712894439698 minutos -> 0.49660277366638184 segundos
Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução
y[0] = 0.0
y[1] = 0.0
y[2] = 0.0
y[3] = 0.0
y[4] = 0.0
y[5] = 0.0
y[6] = -0.0
y[7] = 0.0
y[8] = 0.0
y[9] = 0.0
y[10] = 0.0
y[11] = -0.0
y[12] = 0.0
y[13] = 23.0
y[14] = -0.0
y[15] = 23.0
y[16] = 0.0
y[17] = 0.0
y[18] = -0.0
y[19] = 0.0
y[20] = 0.0
y[21] = -0.0
y[22] = 0.0
y[23] = 0.0
y[24] = 0.0
y[25] = 0.0
y[26] = 23.0
y[27] = 0.0
y[28] = 0.0
y[29] = 0.0
Vértices de L que pertecem a solução do k-mis
z[13] =  1.0
z[15] =  1.0
z[26] =  1.0
Vértices de R que pertencem a solução do kmis
z[30] =  1.0
z[31] =  1.0
z[32] =  1.0
z[34] =  1.0
z[35] =  1.0
z[36] =  1.0
z[37] =  1.0
z[39] =  1.0
z[41] =  1.0
z[42] =  1.0
z[43] =  1.0
z[44] =  1.0


-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=
Valor de k= 3
Valor da probabilidade: 0.5
Tamanho das partições L e R: 40
-=-=-=-=-=-=MEB-V-=-=-=-=-=
Solução ótima encontrada:
Resultado do K-mis: 13.0
Tempo levado: 0.2045111616452535 minutos -> 12.27066969871521 segundos
Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução
y[0] = 0.0
y[1] = 0.0
y[2] = 0.0
y[3] = 0.0
y[4] = 0.0
y[5] = 0.0
y[6] = 0.0
y[7] = 0.0
y[8] = 13.0
y[9] = 0.0
y[10] = 0.0
y[11] = 0.0
y[12] = 0.0
y[13] = 0.0
y[14] = 0.0
y[15] = 0.0
y[16] = 0.0
y[17] = 0.0
y[18] = 13.0
y[19] = 0.0
y[20] = 0.0
y[21] = 0.0
y[22] = 0.0
y[23] = 0.0
y[24] = 0.0
y[25] = 0.0
y[26] = 0.0
y[27] = 0.0
y[28] = 0.0
y[29] = 0.0
y[30] = 0.0
y[31] = 0.0
y[32] = 13.0
y[33] = 0.0
y[34] = 0.0
y[35] = 0.0
y[36] = 0.0
y[37] = 0.0
y[38] = 0.0
y[39] = 0.0
Vértices de L que pertecem a solução do k-mis
z[8] =  1.0
z[18] =  1.0
z[32] =  1.0
Vértices de R que pertencem a solução do kmis
z[40] =  1.0
z[41] =  1.0
z[42] =  1.0
z[44] =  1.0
z[47] = 

-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=
Valor de k= 3
Valor da probabilidade: 0.8
Tamanho das partições L e R: 40
-=-=-=-=-=-=MEB-V-=-=-=-=-=
Solução ótima encontrada:
Resultado do K-mis: 29.000000000000004
Tempo levado: 0.059729393323262533 minutos -> 3.583763599395752 segundos
Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução
y[0] = 0.0
y[1] = 0.0
y[2] = 0.0
y[3] = 0.0
y[4] = 0.0
y[5] = 0.0
y[6] = 0.0
y[7] = 0.0
y[8] = 0.0
y[9] = 0.0
y[10] = 0.0
y[11] = 0.0
y[12] = 0.0
y[13] = -0.0
y[14] = 0.0
y[15] = 0.0
y[16] = 0.0
y[17] = 0.0
y[18] = 0.0
y[19] = 29.0
y[20] = 0.0
y[21] = 0.0
y[22] = 0.0
y[23] = 0.0
y[24] = 0.0
y[25] = 0.0
y[26] = 0.0
y[27] = 0.0
y[28] = 0.0
y[29] = 0.0
y[30] = 29.0
y[31] = 0.0
y[32] = 0.0
y[33] = 0.0
y[34] = 0.0
y[35] = 0.0
y[36] = 0.0
y[37] = 29.0
y[38] = 0.0
y[39] = 0.0
Vértices de L que pertecem a solução do k-mis
z[19] =  1.0
z[30] =  1.0
z[37] =  1.0
Vértices de R que pertencem a solução do kmis
z[40] =  1.0
z[41] =  1.0
z[42] =  1.0
z[4

-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=
Valor de k= 3
Valor da probabilidade: 0.5
Tamanho das partições L e R: 50
-=-=-=-=-=-=MEB-V-=-=-=-=-=
Solução ótima encontrada:
Resultado do K-mis: 16.0
Tempo levado: 0.5758826414744059 minutos -> 34.552958488464355 segundos
Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução
y[0] = 0.0
y[1] = 0.0
y[2] = 0.0
y[3] = 0.0
y[4] = 0.0
y[5] = 0.0
y[6] = 0.0
y[7] = 0.0
y[8] = 0.0
y[9] = 0.0
y[10] = 0.0
y[11] = 0.0
y[12] = 0.0
y[13] = 0.0
y[14] = 0.0
y[15] = 0.0
y[16] = 0.0
y[17] = 0.0
y[18] = 0.0
y[19] = 0.0
y[20] = 0.0
y[21] = 0.0
y[22] = 0.0
y[23] = 0.0
y[24] = 0.0
y[25] = 0.0
y[26] = 0.0
y[27] = 0.0
y[28] = 0.0
y[29] = -0.0
y[30] = 16.0
y[31] = 0.0
y[32] = 0.0
y[33] = 0.0
y[34] = 0.0
y[35] = 16.0
y[36] = -0.0
y[37] = -0.0
y[38] = 0.0
y[39] = 0.0
y[40] = 0.0
y[41] = 0.0
y[42] = 0.0
y[43] = 0.0
y[44] = 16.0
y[45] = 0.0
y[46] = 0.0
y[47] = 0.0
y[48] = 0.0
y[49] = 0.0
Vértices de L que pertecem a solução do k-mis
z[30] =  1.0
z[35] =

-=-=-=-=-=-=Parâmetros passados=-=-=-=-=-=
Valor de k= 3
Valor da probabilidade: 0.8
Tamanho das partições L e R: 50
-=-=-=-=-=-=MEB-V-=-=-=-=-=
Solução ótima encontrada:
Resultado do K-mis: 39.000000000000014
Tempo levado: 0.047004326184590654 minutos -> 2.8202595710754395 segundos
Quantidade de vizinhos de um vértice 'u' em L que pertencem a solução
y[0] = -0.0
y[1] = 0.0
y[2] = 0.0
y[3] = 0.0
y[4] = 39.0
y[5] = 0.0
y[6] = 0.0
y[7] = 0.0
y[8] = -0.0
y[9] = 39.0
y[10] = 0.0
y[11] = 0.0
y[12] = 0.0
y[13] = 0.0
y[14] = 0.0
y[15] = 0.0
y[16] = 0.0
y[17] = 0.0
y[18] = -0.0
y[19] = 0.0
y[20] = 0.0
y[21] = 0.0
y[22] = 0.0
y[23] = 0.0
y[24] = 0.0
y[25] = 0.0
y[26] = 0.0
y[27] = 0.0
y[28] = 0.0
y[29] = 0.0
y[30] = 0.0
y[31] = 39.0
y[32] = 0.0
y[33] = 0.0
y[34] = 0.0
y[35] = 0.0
y[36] = 0.0
y[37] = 0.0
y[38] = 0.0
y[39] = 0.0
y[40] = 0.0
y[41] = 0.0
y[42] = 0.0
y[43] = 0.0
y[44] = 0.0
y[45] = 0.0
y[46] = 0.0
y[47] = 0.0
y[48] = 0.0
y[49] = 0.0
Vértices de L que pertecem a solução do k-mis
z[4]



# Comparação caso-a-caso

|Tamanho  das partições  | probabilidade | Densidade  | resultado MEB-V | tempo MEB-V     | resultado MEB-L |tempo MEB-L   | Melhor |
|------------------------|---------------|------------|-----------------|-----------------|-----------------|--------------|--------|
| 10     | 0.5  | 0.2578947368421053| 4.0 | 0.24039173126220703 | 4.0 | 0.9989292621612549 | MEB-V|
| 10     | 0.8  |0.46842105263157896| 10.0| 0.01565241813659668 | 10.0 | 0.0 | MEB-L |
| 20     | 0.5  |0.2692307692307692| 10.0 |1.243858814239502 | 9.9999 | 8.086018085479736 | MEB- V|
| 20     | 0.8  |0.41794871794871796|17.0| 0.7411541938781738 | 17.0 | 1.243901014328003 | MEB-V|
| 30     | 0.5  |0.2655367231638418|11.0| 12.688663244247437 | 11.0 | 5.373863935470581 | MEB-L |
| 30     | 0.8  |0.41129943502824856|25.0| 0.6771888732910156| 25.0| 2.77099871635437 | MEB-V |
| 40     | 0.5  |0.25253164556962027|11.9999|53.68791580200195|11.9999|17.72377920150757| MEB-L|
| 40     | 0.8  |0.39177215189873416|29.0|10.485508918762207|29.0|12.640235424041748| MEB-V|
| 50     | 0.5  |0.26626262626262626|20.0|94.56442189216614|20.0|15.736402988433838| MEB-L|
| 50     | 0.8  |0.402020202020202|33.9999|149.50166082382202|34.0|23.403944492340088|MEB-L|



# comparação das médias
| Média tensa  MEB-V | Média esparsa MEB-V  | Média tensa  MEB-L | Média esparsa MEB-L  | Média total MEB-V| Média total MEB-L|
|------------------|-----------------|-----------------|-----------------|-----------------|--------------|
|32.284233045578006|32.48505029678345|8.011815929412842|9.583798694610596|32.38464167118073|8.79780731201172|